In [1]:
import torch as t
from mcs import MCS
from PIL import Image

In [2]:
init_state = t.zeros(1,8,8)
init_state[0,4,4] += 1

target_vol = 1.
temperature = t.tensor(27.)
temperature.requires_grad_()

tensor(27., requires_grad=True)

In [16]:
states = [init_state.detach().clone().squeeze().numpy()]
state = init_state.detach().clone()
for i in range(100):
    print(f"-------- MCS {i} --------------")
    state = MCS(state, target_vol, temperature)
    print(state)
    states.append(state.detach().clone().squeeze().numpy())
    if t.sum(state) == 0 or t.sum(state) > 2:
        print("ISSUE DETECTED, STOP SIM",t.sum(state))
        break

imgs = [Image.fromarray((1-state)*255) for state in states]
imgs[0].save("t_1_reflective.gif", save_all=True, append_images=imgs[1:], duration=10, loop=100)

-------- MCS 0 --------------
coordinates of the source pixel on the grid: tensor([3]) tensor([2])
step size: 
 tensor([ 1, -1])
coordinates of the target pixel on the grid: tensor([4]) tensor([1])
vol change tensor([0.])
source is equal to target, no update
tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])
-------- MCS 1 --------------
coordinates of the source pixel on the grid: tensor([4]) tensor([7])
step size: 
 tensor([ 1, -1])
coordinates of the target pixel on the grid: tensor([5]) tensor([6])
vol change tensor([0.])
source is equal to target, no update
tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

In [17]:
print(init_state)
col_idxs = t.arange(8.0).repeat(8, 1) + 1
row_idxs = t.arange(8.0).repeat(8, 1).T + 1

x_mean = t.sum(init_state * col_idxs) / t.sum(init_state)
y_mean = t.sum(init_state * row_idxs) / t.sum(init_state)

print(x_mean, y_mean)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])
tensor(5.) tensor(5.)


In [18]:
print(state)
x_mean_adjusted = t.sum(state * col_idxs) / t.sum(state)
y_mean_adjusted = t.sum(state * row_idxs) / t.sum(state)

print(x_mean_adjusted, y_mean_adjusted)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]], grad_fn=<CopySlices>)
tensor(4.5000, grad_fn=<DivBackward0>) tensor(5., grad_fn=<DivBackward0>)


In [19]:
dist = t.sqrt((x_mean - x_mean_adjusted)**2 + (y_mean-y_mean_adjusted)**2)
print(dist)

tensor(0.5000, grad_fn=<SqrtBackward0>)


In [7]:
t.autograd.grad(dist, temperature)

(tensor(0.),)

In [20]:
t.autograd.grad(t.sum((state - init_state)**2), temperature)

(tensor(0.0026),)